# 1 - Basics run

**Objectives:**
1. Create a fixed tilt system 
2. Sample the Ground
3. Generate plots 

## 1. Setup and Create PV ICE Simulation Object

In [1]:
# Uncomment the below code to install the environment in the Google Collab journal
'''
%%bash
wget https://github.com/LBNL-ETA/Radiance/releases/download/012cb178/Radiance_012cb178_Linux.zip -O radiance.zip
unzip radiance.zip 
tar -xvf radiance-5.3.012cb17835-Linux.tar.gz
ls -l $PWD
!pip install git+https://github.com/NREL/bifacial_radiance.git@development
'''

'\n%%bash\nwget https://github.com/LBNL-ETA/Radiance/releases/download/012cb178/Radiance_012cb178_Linux.zip -O radiance.zip\nunzip radiance.zip \ntar -xvf radiance-5.3.012cb17835-Linux.tar.gz\nls -l $PWD\n!pip install git+https://github.com/NREL/bifacial_radiance.git@development\n'

In [2]:
import bifacial_radiance as br
import numpy as np
import pandas as pd

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
bifacial_radiance version  0.4.2+210.g99a55d7


In [4]:
%env PATH="$PATH:/radiance-5.3.012cb17835-Linux/usr/local/radiance/bin"
%env LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/radiance-5.3.012cb17835-Linux/usr/local/radiance/lib"
%env RAYPATH="$RAYPATH:/radiance-5.3.012cb17835-Linux/usr/local/radiance/lib"

#echo "/radiance-5.3.012cb17835-Linux/usr/local/radiance/bin" >> $GITHUB_PATH

env: PATH="$PATH:/radiance-5.3.012cb17835-Linux/usr/local/radiance/bin"
env: LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/radiance-5.3.012cb17835-Linux/usr/local/radiance/lib"
env: RAYPATH="$RAYPATH:/radiance-5.3.012cb17835-Linux/usr/local/radiance/lib"


In [5]:
import os
from pathlib import Path

testfolder = 'TEMP'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in TEMP


In [6]:
simulationname = 'tutorial_1'

# Location:
lat = 39.7407 # ° N, 
lon = -105.1686 # ° W


# Scene Parameters:
azimuth_ang=90 # Facing south
tilt = 30 # tilt.

# MakeModule Parameters
moduletype='test-module'
numpanels = 1  # AgriPV site has 3 modules along the y direction
module_x = 2 # m
module_y = 1 # m. slope we will measure
sensorsy=2
sensorsground=5

# SceneDict Parameters
pitch = 6 # m
albedo = 0.2  #'grass'     # ground albedo
clearance_height = 0.5 # m  
nMods = 4 # six modules per row.
nRows = 3  # 3 row


In [7]:
demo = br.RadianceObj(simulationname,path = testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) # NJ lat/lon 40.0583° N, 74.4057


path = TEMP
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


Getting weather file: USA_CO_Golden-NREL.724666_TMY3.epw


 ... OK!


In [8]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: test-module
Module test-module updated in module.json


In [9]:
#Valid options: mm_dd, mm_dd_HH, mm_dd_HHMM, YYYY-mm-dd_HHMM
metdata = demo.readWeatherFile(epwfile, coerce_year=2021, starttime='2021-06-01', endtime='2021-06-30')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021


Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [10]:
demo.genCumSky()  
#demo.gendaylit(timeindex=0)  

Loaded  EPWs\metdata_temp.csv
message: 'gencumulativesky' is not recognized as an internal or external command,
operable program or batch file.


'skies\\cumulative.rad'

In [11]:
sceneDict = {'tilt':tilt,'pitch': pitch,'clearance_height':clearance_height,'azimuth':azimuth_ang, 
             'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(module=moduletype, sceneDict=sceneDict) 


In [12]:
octfile = demo.makeOct()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
analysis = br.AnalysisObj(octfile, demo.name)  
frontscan, backscan, groundscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy, sensorsground=sensorsground)


In [ ]:
analysis.analysis(octfile, simulationname+"_groundscan_East", groundscan, backscan)  # compare the back vs front irradiance  